## ਲੋੜੀਂਦੇ ਲਾਇਬ੍ਰੇਰੀਆਂ ਇਮਪੋਰਟ ਕਰੋ  
ਫਾਈ-4 ਮਾਡਲ ਨੂੰ ਲੋਡ ਅਤੇ ਵਰਤਣ ਲਈ ਲੋੜੀਂਦੇ PyTorch ਅਤੇ transformers ਲਾਇਬ੍ਰੇਰੀਆਂ ਨੂੰ ਇਮਪੋਰਟ ਕਰੋ।  


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

## ਰੈਂਡਮ ਸੀਡ ਸੈਟ ਕਰੋ  
ਵੱਖ-ਵੱਖ ਰਨਜ਼ ਵਿੱਚ ਦੁਹਰਾਏ ਜਾ ਸਕਣ ਵਾਲੇ ਨਤੀਜੇ ਯਕੀਨੀ ਬਣਾਉਣ ਲਈ ਰੈਂਡਮ ਸੀਡ ਸੈਟ ਕਰੋ।


In [ ]:
torch.random.manual_seed(0)

## ਫਾਈ-4-ਮਿਨੀ-ਫਲੈਸ਼-ਰੀਜ਼ਨਿੰਗ ਮਾਡਲ ਅਤੇ ਟੋਕਨਾਈਜ਼ਰ ਲੋਡ ਕਰੋ  
ਮਾਈਕਰੋਸਾਫਟ ਫਾਈ-4-ਮਿਨੀ-ਫਲੈਸ਼-ਰੀਜ਼ਨਿੰਗ ਮਾਡਲ ਅਤੇ ਇਸ ਨਾਲ ਸੰਬੰਧਿਤ ਟੋਕਨਾਈਜ਼ਰ ਨੂੰ ਹੱਗਿੰਗ ਫੇਸ ਤੋਂ ਲੋਡ ਕਰੋ। ਮਾਡਲ ਨੂੰ ਤੇਜ਼ ਨਤੀਜਿਆਂ ਲਈ CUDA 'ਤੇ ਲੋਡ ਕੀਤਾ ਜਾਵੇਗਾ।  


In [ ]:
model_id = "microsoft/Phi-4-mini-flash-reasoning"
model = AutoModelForCausalLM.from_pretrained(
   model_id,
   device_map="cuda",
   torch_dtype="auto",
   trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

## ਇਨਪੁਟ ਸੁਨੇਹਾ ਤਿਆਰ ਕਰੋ  
ਮਾਡਲ ਲਈ ਚੈਟ ਟੈਂਪਲੇਟ ਦੀ ਵਰਤੋਂ ਕਰਕੇ ਗੁਣਾਤਮਕ ਸਮੀਕਰਨ ਗਣਿਤ ਸਮੱਸਿਆ ਵਾਲਾ ਗੱਲਬਾਤ ਸੁਨੇਹਾ ਬਣਾਓ।


In [ ]:
messages = [{
   "role": "user",
   "content": "How to solve 3*x^2+4*x+5=1?"
}]   
inputs = tokenizer.apply_chat_template(
   messages,
   add_generation_prompt=True,
   return_dict=True,
   return_tensors="pt",
)

## ਜਵਾਬ ਤਿਆਰ ਕਰੋ
ਦਿੱਤੇ ਗਏ ਪੈਰਾਮੀਟਰਾਂ ਜਿਵੇਂ ਕਿ temperature ਅਤੇ top_p ਦੀ ਵਰਤੋਂ ਕਰਕੇ ਮਾਡਲ ਤੋਂ ਨਤੀਜਾ ਪ੍ਰਾਪਤ ਕਰੋ, ਤਾਂ ਜੋ ਨਤੀਜੇ ਵਿੱਚ ਨਿਯੰਤਰਿਤ randomness ਹੋਵੇ।


In [ ]:
outputs = model.generate(
   **inputs.to(model.device),
   max_new_tokens=32768,
   temperature=0.6,
   top_p=0.95,
   do_sample=True,
)

## ਆਉਟਪੁੱਟ ਨੂੰ ਟੈਕਸਟ ਵਿੱਚ ਡਿਕੋਡ ਕਰੋ
ਜਨਰੇਟ ਕੀਤੇ ਟੋਕਨ ਸੀਕਵੰਸ ਨੂੰ ਵਾਪਸ ਮਨੁੱਖ-ਪੜ੍ਹਨਯੋਗ ਟੈਕਸਟ ਵਿੱਚ ਬਦਲੋ, ਮੂਲ ਇਨਪੁਟ ਟੋਕਨ ਨੂੰ ਛੱਡ ਕੇ ਸਿਰਫ ਮਾਡਲ ਦੀ ਪ੍ਰਤੀਕ੍ਰਿਆ ਦਿਖਾਓ।


In [ ]:
outputs = tokenizer.batch_decode(outputs[:, inputs["input_ids"].shape[-1]:])


---

**ਅਸਵੀਕਰਤੀ**:  
ਇਹ ਦਸਤਾਵੇਜ਼ AI ਅਨੁਵਾਦ ਸੇਵਾ [Co-op Translator](https://github.com/Azure/co-op-translator) ਦੀ ਵਰਤੋਂ ਕਰਕੇ ਅਨੁਵਾਦ ਕੀਤਾ ਗਿਆ ਹੈ। ਹਾਲਾਂਕਿ ਅਸੀਂ ਸਹੀ ਹੋਣ ਦੀ ਕੋਸ਼ਿਸ਼ ਕਰਦੇ ਹਾਂ, ਕਿਰਪਾ ਕਰਕੇ ਧਿਆਨ ਦਿਓ ਕਿ ਸਵੈਚਾਲਿਤ ਅਨੁਵਾਦਾਂ ਵਿੱਚ ਗਲਤੀਆਂ ਜਾਂ ਅਸੁੱਤੀਆਂ ਹੋ ਸਕਦੀਆਂ ਹਨ। ਮੂਲ ਦਸਤਾਵੇਜ਼, ਜੋ ਇਸਦੀ ਮੂਲ ਭਾਸ਼ਾ ਵਿੱਚ ਹੈ, ਨੂੰ ਅਧਿਕਾਰਤ ਸਰੋਤ ਮੰਨਿਆ ਜਾਣਾ ਚਾਹੀਦਾ ਹੈ। ਮਹੱਤਵਪੂਰਨ ਜਾਣਕਾਰੀ ਲਈ, ਪੇਸ਼ੇਵਰ ਮਨੁੱਖੀ ਅਨੁਵਾਦ ਦੀ ਸਿਫਾਰਸ਼ ਕੀਤੀ ਜਾਂਦੀ ਹੈ। ਇਸ ਅਨੁਵਾਦ ਦੀ ਵਰਤੋਂ ਤੋਂ ਪੈਦਾ ਹੋਣ ਵਾਲੇ ਕਿਸੇ ਵੀ ਗਲਤਫਹਿਮੀ ਜਾਂ ਗਲਤ ਵਿਆਖਿਆ ਲਈ ਅਸੀਂ ਜ਼ਿੰਮੇਵਾਰ ਨਹੀਂ ਹਾਂ।
